<a href="https://colab.research.google.com/github/yeesem/Time-Series-Data-Analysis/blob/main/Walk_Forward_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -nc https://lazyprogrammer.me/course_files/airline_passengers.csv

In [ ]:
!pip install -U statsmodels

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import itertools

In [5]:
 from sklearn.metrics import mean_squared_error
 from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [9]:
df = pd.read_csv('/content/airline_passengers.csv',index_col = 'Month',parse_dates = True)

In [10]:
df.index.freq = 'MS'

In [11]:
df.shape

(144, 1)

In [12]:
#Assume the forecast horizon we care about is 12
#Validate over 10 steps
h = 12
steps = 10
Ntest = len(df) - h - steps + 1

In [13]:
#Configuration hyperparameters to try
trend_type_list = ['add','mul']
seasonal_type_list = ['add','mul']
damped_trend_list = [True,False]
init_method_list = ['estimated','heuristic','legacy-heuristic']
use_boxcox_list = [True,False,0]